# Fine-tuning by KorQuAD

### 데이터셋 준비

In [24]:
!pip install gdown

In [25]:
!gdown https://raw.githubusercontent.com/korquad/korquad.github.io/refs/heads/master/dataset/KorQuAD_v1.0_dev.json

Downloading...
From: https://raw.githubusercontent.com/korquad/korquad.github.io/refs/heads/master/dataset/KorQuAD_v1.0_dev.json
To: /Users/woojin/Desktop/SK Networks Family AI Camp_17/SKN-17-Family-AI-Camp/LLM/07_openai_finetuning/KorQuAD_v1.0_dev.json
3.88MB [00:00, 159MB/s]                                                         


### 데이터 준비

In [26]:
import json

with open('KorQuAD_v1.0_dev.json', 'r', encoding='utf-8') as f:
    dev_data = json.load(f)
    
dev_data['data'][0]

{'paragraphs': [{'qas': [{'answers': [{'text': '1989년 2월 15일',
       'answer_start': 0}],
     'id': '6548850-0-0',
     'question': '임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배 된 날은?'},
    {'answers': [{'text': '임수경', 'answer_start': 125}],
     'id': '6548850-0-1',
     'question': '1989년 6월 30일 평양축전에 대표로 파견 된 인물은?'},
    {'answers': [{'text': '1989년', 'answer_start': 0}],
     'id': '6548853-0-0',
     'question': '임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배된 연도는?'},
    {'answers': [{'text': '학생회관 건물 계단', 'answer_start': 365}],
     'id': '6548853-0-1',
     'question': '임종석을 검거한 장소는 경희대 내 어디인가?'},
    {'answers': [{'text': '서울지방경찰청 공안분실', 'answer_start': 457}],
     'id': '6548853-0-2',
     'question': '임종석이 조사를 받은 뒤 인계된 곳은 어딘가?'},
    {'answers': [{'text': '임종석', 'answer_start': 87}],
     'id': '6332405-0-0',
     'question': '1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의로 지명수배된 사람의 이름은?'},
    {'answers': [{'text': '여의도 농민 폭력 시위', 'answer_start': 13}],
     'id': '6332405-0-1',
     'question': '임종석이 1989년 2월 

In [27]:
items = [item for topic in dev_data['data'] for item in topic['paragraphs']]
items[0]

{'qas': [{'answers': [{'text': '1989년 2월 15일', 'answer_start': 0}],
   'id': '6548850-0-0',
   'question': '임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배 된 날은?'},
  {'answers': [{'text': '임수경', 'answer_start': 125}],
   'id': '6548850-0-1',
   'question': '1989년 6월 30일 평양축전에 대표로 파견 된 인물은?'},
  {'answers': [{'text': '1989년', 'answer_start': 0}],
   'id': '6548853-0-0',
   'question': '임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배된 연도는?'},
  {'answers': [{'text': '학생회관 건물 계단', 'answer_start': 365}],
   'id': '6548853-0-1',
   'question': '임종석을 검거한 장소는 경희대 내 어디인가?'},
  {'answers': [{'text': '서울지방경찰청 공안분실', 'answer_start': 457}],
   'id': '6548853-0-2',
   'question': '임종석이 조사를 받은 뒤 인계된 곳은 어딘가?'},
  {'answers': [{'text': '임종석', 'answer_start': 87}],
   'id': '6332405-0-0',
   'question': '1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의로 지명수배된 사람의 이름은?'},
  {'answers': [{'text': '여의도 농민 폭력 시위', 'answer_start': 13}],
   'id': '6332405-0-1',
   'question': '임종석이 1989년 2월 15일에 지명수배 받은 혐의는 어떤 시위를 주도했다는 것인가?'}],
 'context': '1989년 2월 15

In [28]:
qa_dict = {}

for item in items:
    for entry in item['qas']:
        question = entry['question']
        answer = [ans['text'] for ans in entry['answers']][0]
        qa_dict[question] = answer
        
len(qa_dict)

5764

In [29]:
qa_dict = dict(list(qa_dict.items())[:20])
qa_dict

{'임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배 된 날은?': '1989년 2월 15일',
 '1989년 6월 30일 평양축전에 대표로 파견 된 인물은?': '임수경',
 '임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배된 연도는?': '1989년',
 '임종석을 검거한 장소는 경희대 내 어디인가?': '학생회관 건물 계단',
 '임종석이 조사를 받은 뒤 인계된 곳은 어딘가?': '서울지방경찰청 공안분실',
 '1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의로 지명수배된 사람의 이름은?': '임종석',
 '임종석이 1989년 2월 15일에 지명수배 받은 혐의는 어떤 시위를 주도했다는 것인가?': '여의도 농민 폭력 시위',
 '정부의 헌법개정안 준비 과정에 대해서 청와대 비서실이 아니라 국무회의 중심으로 이뤄졌어야 했다고 지적한 원로 헌법학자는?': '허영',
 "'행보가 비서 본연의 역할을 벗어난다', '장관들과 내각이 소외되고 대통령비서실의 권한이 너무 크다'는 의견이 제기된 대표적인 예는?": '10차 개헌안 발표',
 '국무회의의 심의를 거쳐야 한다는 헌법 제 몇 조의 내용인가?': '제89조',
 '법무부 장관을 제쳐놓고 민정수석이 개정안을 설명하는 게 이해가 안 된다고 지적한 경희대 석좌교수 이름은?': '허영',
 '미국 군대 내 두번째로 높은 직위는 무엇인가?': '미국 육군 부참모 총장',
 '로널드 레이건 정부 출범 당시 알렉산더 헤이그는 어떤 직책을 맡았는가?': '초대 국무장관직',
 '알렉산더 헤이그는 어느 대통령의 밑에서 국무장관을 지냈는가?': '로널드 레이건 대통령',
 '로널드 레이건 대통령 밑에서 일한 국무 장관은 누구인가?': '알렉산더 메이그스 헤이그 2세',
 '미국 군대에서 두번째로 높은 직위는?': '미국 육군 부참모 총장',
 '알렉산더 메이그스 헤이그의 생년월일은?': '1924년 12월 2일',
 '알렉산더 헤이그가 로널드 레이건 대통령 밑에서 맡은 직책은 무엇이었나?': '국무장

In [30]:
results = []

for q, a in qa_dict.items():
    qa = {
        "message": [
            {"role": "system", "content": ""},
            {"role": "user", "content": q},
            {"role": "assistant", "content": a}
        ]
    }
    results.append(qa)

results

[{'message': [{'role': 'system', 'content': ''},
   {'role': 'user', 'content': '임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배 된 날은?'},
   {'role': 'assistant', 'content': '1989년 2월 15일'}]},
 {'message': [{'role': 'system', 'content': ''},
   {'role': 'user', 'content': '1989년 6월 30일 평양축전에 대표로 파견 된 인물은?'},
   {'role': 'assistant', 'content': '임수경'}]},
 {'message': [{'role': 'system', 'content': ''},
   {'role': 'user', 'content': '임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배된 연도는?'},
   {'role': 'assistant', 'content': '1989년'}]},
 {'message': [{'role': 'system', 'content': ''},
   {'role': 'user', 'content': '임종석을 검거한 장소는 경희대 내 어디인가?'},
   {'role': 'assistant', 'content': '학생회관 건물 계단'}]},
 {'message': [{'role': 'system', 'content': ''},
   {'role': 'user', 'content': '임종석이 조사를 받은 뒤 인계된 곳은 어딘가?'},
   {'role': 'assistant', 'content': '서울지방경찰청 공안분실'}]},
 {'message': [{'role': 'system', 'content': ''},
   {'role': 'user',
    'content': '1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의로 지명수배된 사람의 이름은?'},
   {'role': 'assistant', 

In [31]:
with open('korquad_data.jsonl', 'w', encoding='utf-8') as f:
    for qa in results:
        json_str = json.dumps(qa, ensure_ascii=False)
        f.write(json_str + '\n')

### Fine-tuned 모델 추론

In [32]:
from dotenv import load_dotenv
load_dotenv()

True

In [33]:
prompt = "법무부 장관을 제쳐놓고 민정수석이 개정안을 설명하는 게 이해가 안 된다고 지적한 경희대 석좌교수 이름은?"

In [34]:
from openai import OpenAI

base_model = "gpt-3.5-turbo-0125"

client = OpenAI()

response = client.chat.completions.create(
    model=base_model,
    messages=[
        {"role": "user", "content": ""},
        {"role": "user", "content": prompt}
    ]
)

In [37]:
from openai import OpenAI

ft_model = "ft:gpt-3.5-turbo-0125:skn17:korquad:CDiBQ6m6"

client = OpenAI()

ft_response = client.chat.completions.create(
    model=ft_model,
    messages=[
        {"role": "user", "content": ""},
        {"role": "user", "content": prompt}
    ]
)

In [39]:
print("기본 모델:")
print(response.choices[0].message.content)
print()
print("파인튜닝 모델:")
print(ft_response.choices[0].message.content)

기본 모델:
김종대 교수 입니다.

파인튜닝 모델:
김재철
